# 3. Probabilité

### Question 1 :

$$ Q(d,k) = \sum_{j=2}^{6} \frac{Q(d-1,k-j)}{5}$$

Le cas d'initialisation est $Q(1,k)=\frac{1}{5}$ avec $2 \leq k\leq6$

On a $$P(1,k) = \frac{5}{6}Q(1,k) = \frac{1}{6}$$ 


In [11]:
import numpy as np
import pandas as pd

rows=200
columns=200
pd.set_option('display.max_rows',rows)
pd.set_option('display.max_columns',columns)
pd.set_option('display.width', 1000)

In [12]:
def tab_Pdk(dmax): #Retourne le tableau des P(d,k) pour d et k supérieur à 1
    
    kmax = dmax*6  
    res = np.ones((dmax, kmax))
    
   
    for d in range(dmax):
        for k in range(kmax):
            if (k+1)<2*(1+d) or (k+1)>6*(d+1):
                res[d,k] = 0
                
    #Cas d=1
    for i in range(1,6):
        res[0][i] = 1/5
    
    #on met les valeurs des Q(d,k) dans toutes les cases non nulles
    for d in range(1,dmax):
        for k in range(kmax):
            if (res[d,k]==1) :
                res[d,k] = 0
                #on fait un for dans les valeurs qui sont réalisables. 
                #le +1 après le min est là car nous sommes dans un range
                for i in range(max(k-6,2*(d+1-1)-1) , min(k-2,6*(d+1-1))+1):
                    res[d,k] += res[d-1,i]/5
    
    #On multiplie toutes les cases selon la formule pour obtenir les P(d,k)
    for d in range(dmax):
        for k in range(kmax) :
            res[d,k] =  res[d,k]*(5/6)**(d+1)
            
    for d in range(dmax):
        res[d, 0] = 1-(5/6)**(d+1)
            
    return res

In [44]:
nb_dice = 3
pdk = tab_Pdk(nb_dice)
pd.DataFrame(pdk, #les lignes suivantes ne servent qu'à clarifier le tab
             index = [i+1 for i in range(nb_dice)] ,      
             columns=[i+1 for i in range(6*nb_dice)] )

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.305556,0.000000,0.000000,0.027778,0.055556,0.083333,0.111111,0.138889,0.111111,0.083333,0.055556,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.421296,0.000000,0.000000,0.000000,0.000000,0.004630,0.013889,0.027778,0.046296,0.069444,0.083333,0.087963,0.083333,0.069444,0.046296,0.027778,0.013889,0.00463


# 4. Variante séquentielle

## 4.1 Stratégie aveugle

Soit $d^*(D)$ le nombre de dés maximisant l'espérance du nombre de points. Nous appellerons dans la suite **stratégie aveugle**, la stratégie consistant à lancer $d^∗(D)$ dés.

Éspérance du nombre du points obtenus pour $d$ dés :
$$EP(d) = 4d\biggl(\frac{5}{6}\biggr)^d+1-\biggl(\frac{5}{6}\biggr)^d$$

In [30]:
def Q2(D) : #Retourne le nombre optimale de dés à jouer pour maximiser l'ésperance de nombre de points d*(D)
    esperance = []
    for d in range(1,D+1):
        esperance.append(4*d*(5/6)**d+1-(5/6)**d)
    return np.argmax(esperance)

### Question 3 :
Voici deux cas où la *stratégie aveugle* n'est pas toujours la meilleur:
- Le cas où un joueur est à deux points du score optimal et qu'il possède 6 dés, $d^*(6)=5$ or il serait plus judicieux de ne lancer que un ou deux dés.

- Le cas où deux joueurs jouent avec 3 dés. Un joueur A n'est qu'à $1$ point de la victoire et que c'est au joueur B de jouer, qui est lui à 17 points de la victoire. On a $d^*(3)=2$ pourtant si le joueur B ne joue pas $3$ dés il n'a aucune chance de gagner.

## 4.2 Programmation dynamique

Soit $(i, j)$ l'état où le premier joueur a cumulé $i$ points, le deuxième joueur a cumulé $j$ points, et c'est au joueur $1$ de jouer.

Soit $EG(i, j)$ l'espérance de gain du joueur $1$ dans l'état $(i, j)$,
en supposant que son adversaire et lui-même jouent de façon optimale.

### Question 4 :
 Calcule de l'espérance de gain $EG(i, j)$ pour le joueur 1 à un état $(i, j)$ donné.

In [31]:
def EG1(i,j,cpt):
    global DOPT
    if (j>=N):
        return (-1,cpt)
    if (i>=N-1):
        DOPT[i,j]=1
        return (1,cpt)
    if (E1[i,j]!=0):
        return (E1[i,j],cpt)
    m=-1
    for d in range(D):
        sum=0
        for k in range(1,6*D):
            sum+=EG2(i+k,j,cpt+1)[0]*P[d,k-1]
        if (sum>m):
            #print("(i,j): "+str(i)+' '+str(j))
            m=sum
            if (cpt==0):
                #print("d: "+str(d))
                #print("sum: "+str(sum))
                tabdes.append(d)
            DOPT[i,j]=d+1

                
    E1[i,j]=m
    return (m,cpt)
    
de2=-1
def EG2(i,j,cpt):
    if (i>=N):
        return (1,cpt+1)
    if (j>=N-1):
        return (-1,cpt+1)
    if (E2[i,j]!=0):
        return (E2[i,j],cpt+1)
    m=1
    for d2 in range(D):
        sum=0
        for k2 in range(1,6*D):
            sum+=EG1(i,j+k2,cpt+1)[0]*P[d2,k2-1]
        if (sum<m):
            m=sum
    E2[i,j]=m
    return (m,cpt+1)




In [32]:
def EG_J1(i,j):
    return (EG1(i,j,0),tabdes[-1]+1)



In [33]:
import matplotlib.pyplot as plt

def strategie_optimale(score,adversaire):
    global DOPT
    return int(DOPT[score][adversaire])



In [34]:
import random


def simulation(n,de):
    sommea=0
    sommeb=0
    while (sommea<n and sommeb<n):
        sommetempa=[]
        sommetempb=[]
        #print("Score a: {}, score b: {}".format(sommea,sommeb))
        #dicemaxa=strategie_optimale(sommea,sommeb) 
        dicemaxa=strategie_optimale(sommea,sommeb) 
        dicemaxb=Q2(de)
        
        #print("A joue {} dés".format(dicemaxa))
        #print("B joue {} dés".format(dicemaxb))
        for k in range(dicemaxa):
            dice=random.randint(1,6)
            sommetempa.append(dice)
            #print("Le joueur A a tiré un {}".format(dice))
        for k in range(dicemaxb):
            dice=random.randint(1,6)
            #print("Le joueur B a tiré un {}".format(dice))
            sommetempb.append(dice)
        if (1 in sommetempa):
            sommea+=1
        else:
            sommea+=np.sum(sommetempa)
        if (1 in sommetempb):
            sommeb+=1
        else:
            sommeb+=np.sum(sommetempb)
    #print("SCORE FINAL: Score a: {}, score b: {}".format(sommea,sommeb))
    if (sommea>sommeb):
        return 1
    if (sommea<sommeb):
        return 2
    if (sommea==sommeb):
        return 0
    return


In [35]:

def evaluationD(Dmax,Nmax):
    global N
    global E1
    global E2
    global tabdes
    global P
    global DOPT
    global D
    N=Nmax
    plota=[]
    plotb=[]
    for dice in range(1,Dmax):
        D=dice
        P = tab_Pdk(D)
        tabdes=[0]
        E1=np.zeros((Nmax+1,Nmax+1))
        E1[:,-1]=-1
        E1[-1,:]=1
        E1[-2,:]=1
        E2=np.zeros((Nmax+1,Nmax+1))
        E2[-1,:]=1
        E2[:,-1]=-1
        DOPT=np.zeros((Nmax+1,Nmax+1))
        EG_J1(0,0)
        for i in range(Nmax):
            EG_J1(0,i)
            EG_J1(i,0)
        cpta=0
        cptb=0
        cptc=0
        for k in range(100):
            a=simulation(Nmax,dice)
            if (a==1):
                cpta+=1
            if (a==2):
                cptb+=1
            if (a==0):
                cptc+=1
        plota.append(cpta/100)
        plotb.append(cptb/100)
    plt.plot([i for i in range (1,Dmax)],plota,label='Joueur A, stratégie aléatoire')
    plt.plot([i for i in range (1,Dmax)],plotb,label='Joueur B, stratégie optimale')
    plt.xlabel("Nombre de dés disponibles")
    plt.ylabel("Probabilité de victoire")
    plt.title("Probabilité de victoire en fonction du nombre de dés disponibles")
    plt.legend()
    plt.show()
    
def evaluationN(Dmax,Nmax):
    global N
    global E1
    global E2
    global tabdes
    global P
    global DOPT
    global D
    N=Nmax
    plota=[]
    plotb=[]
    for score in range(1,Nmax):
        N=score
        P = tab_Pdk(Dmax)
        tabdes=[0]
        E1=np.zeros((Nmax+1,Nmax+1))
        E1[:,-1]=-1
        E1[-1,:]=1
        E1[-2,:]=1
        E2=np.zeros((Nmax+1,Nmax+1))
        E2[-1,:]=1
        E2[:,-1]=-1
        DOPT=np.zeros((Nmax+1,Nmax+1))
        EG_J1(0,0)
        for i in range(Nmax):
            EG_J1(0,i)
            EG_J1(i,0)
        cpta=0
        cptb=0
        cptc=0
        for k in range(1000):
            a=simulation(score,Dmax)
            if (a==1):
                cpta+=1
            if (a==2):
                cptb+=1
            if (a==0):
                cptc+=1
        plota.append(cpta/1000)
        plotb.append(cptb/1000)
    plt.plot([i for i in range (1,Nmax)],plota,label='Joueur A, stratégie optimale')
    plt.plot([i for i in range (1,Nmax)],plotb,label='Joueur B, stratégie aveugle')
    plt.xlabel("Valeur de N")
    plt.ylabel("Probabilité de victoire")
    plt.title("Probabilité de victoire en fonction de N pour "+str(Dmax)+" dés")
    plt.legend()
    plt.show()
    

# 5. Variante séquentielle

## 5.1 Jeu en un coup

On considère en première approche une version simplifuiée où on lance une fois les dés.
Le gagnant est simplement celui qui remporte le plus de points.
On note  $EG1(d1, d2)$ l'espérance de gain du joueur 1 s'il lance $d1$ dés a lorsque le joueur2 en lance $d2$.

### Question 10 :

  Calcul de $EG1(d1, d2)$ en fonction des probabilités $P(d, k)$

In [49]:
def Eg1(d1,d2):
    return (sum([sum([pdk[d1-1,k1-1]*pdk[d2-1,k2-1] for k1 in range(1,6*d1+1) if k1>k2 ])
                for k2 in range(1,6*d2+1)])
            -sum([sum([pdk[d1-1,k1-1]*pdk[d2-1,k2-1] for k1 in range(1,6*d1+1) if k1<k2 ])
                  for k2 in range(1,6*d2+1)])
           )
Eg1(3,3)    

-5.551115123125783e-17

In [50]:
EG=[[Eg1(d1,d2) for d1 in range(1,nb_dice+1)] for d2 in range(1,nb_dice+1)]
EG

[[0.0, 0.3750000000000003, 0.2268518518518522],
 [-0.3750000000000003, 0.0, 0.19881687242798363],
 [-0.22685185185185208, -0.19881687242798368, -5.551115123125783e-17]]

### Question 12 :

 PL pour obtenir une stratégie optimale pour J1

In [51]:
from pulp import *

def opt_simultane(EG,nb_dice):
    var=['z']+[i for i in range(1,nb_dice+1)]

    # Create the 'prob' variable to contain the problem data
    prob = LpProblem("The opt strategy for player 1", LpMaximize)

    # A dictionary called 'dec_vars' is created to contain the referenced Variables
    dec_vars = LpVariable.dicts("Dec",var[1:],0)
    dec_vars.update(LpVariable.dicts("Dec",var[0]))

    # The objective function is added to 'prob' first
    prob += lpSum([dec_vars['z']]),"objective function"
    
    coef=dict()
    coef['z']=1
    
    for i in var[1:]:
        coef[i]=-1
    
    for j in range(1,nb_dice+1):
        prob += lpSum([coef['z']*dec_vars['z']]+[EG[i-1][j-1]*dec_vars[i] for i in var[1:]]) <= 0.0, "{} contrast".format(j)

    prob += lpSum([dec_vars[i] for i in var[1:]]) == 1
    
    # The problem data is written to an .lp file
    prob.writeLP("strategy_player1.lp")

    # The problem is solved using PuLP's choice of Solver
    prob.solve()

    # The status of the solution is printed to the screen
    print ("Status:", LpStatus[prob.status])

    # Each of the variables is printed with it's resolved optimum value
    print(type(prob.variables()))
    for v in prob.variables():
        print(v.name, "=", v.varValue)

    # The optimised objective function value is printed to the screen    
    print("Esperance de gain optimal = ", value(prob.objective))
    
    strategy=[v.varValue for v in prob.variables() if v.name != "Dec_z"]
    return strategy

print(opt_simultane(EG,nb_dice))


Status: Optimal
<class 'list'>
Dec_1 = 0.0
Dec_2 = 0.0
Dec_3 = 1.0
Dec_z = 0.0
Esperance de gain optimal =  0.0
[0.0, 0.0, 1.0]
